Copyright (c) Microsoft Corporation. All rights reserved.  
Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/NotebookVM/how-to-use-azureml/machine-learning-pipelines/intro-to-pipelines/aml-pipelines-with-automated-machine-learning-step.png)

# Azure Machine Learning Pipeline with AutoMLStep (Udacity Project 2)


## Azure Machine Learning and Pipeline SDK-specific imports

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.19.0


## Initialize Workspace
Initialize a workspace object from persisted configuration. Make sure the config file is present at .\config.json

In [2]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

quick-starts-ws-135932
aml-quickstarts-135932
southcentralus
3e42d11f-d64d-4173-af9b-12ecaa1030b3


## Create an Azure ML experiment
Let's create an experiment named "automlstep-classification" and a folder to hold the training scripts. The script runs will be recorded under the experiment in Azure.

The best practice is to use separate folders for scripts and its dependent files for each step and specify that folder as the `source_directory` for the step. This helps reduce the size of the snapshot created for the step (only the specific folder is snapshotted). Since changes in any files in the `source_directory` would trigger a re-upload of the snapshot, this helps keep the reuse of the step when there are no changes in the `source_directory` of the step.

*Udacity Note:* There is no need to create an Azure ML experiment, this needs to re-use the experiment that was already created


In [3]:
# Choose a name for the run history container in the workspace.
# NOTE: update these to match your existing experiment name
experiment_name = 'sayed-bankmarketing-exp'
project_folder = './pipeline-bankmarketing-project'

experiment = Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
sayed-bankmarketing-exp,quick-starts-ws-135932,Link to Azure Machine Learning studio,Link to Documentation


### Create or Attach an AmlCompute cluster
You will need to create a [compute target](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#compute-target) for your AutoML run. In this tutorial, you get the default `AmlCompute` as your training compute resource.

**Udacity Note** There is no need to create a new compute target, it can re-use the previous cluster

In [4]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# Choosing a name for the CPU cluster
amlcompute_cluster_name = "sayed-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 3)
# For a more detailed view of current AmlCompute status, use get_status().

Creating
Succeeded...............................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


In [5]:
compute_target

AmlCompute(workspace=Workspace.create(name='quick-starts-ws-135932', subscription_id='3e42d11f-d64d-4173-af9b-12ecaa1030b3', resource_group='aml-quickstarts-135932'), name=sayed-cluster, id=/subscriptions/3e42d11f-d64d-4173-af9b-12ecaa1030b3/resourceGroups/aml-quickstarts-135932/providers/Microsoft.MachineLearningServices/workspaces/quick-starts-ws-135932/computes/sayed-cluster, type=AmlCompute, provisioning_state=Succeeded, location=southcentralus, tags=None)

## Data

**Udacity note:** Make sure the `key` is the same name as the dataset that is uploaded, and that the description matches. If it is hard to find or unknown, loop over the `ws.datasets.keys()` and `print()` them.
If it *isn't* found because it was deleted, it can be recreated with the link that has the CSV 

In [6]:
# Loading the dataset from the Workspace. or creating it from the file
found = False
key = "bankmarketing_dataset_sayed_ahmed"
description_text = "Bankmarketing DataSet for Udacity Project 2"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        bank_data = 'https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'
        dataset = Dataset.Tabular.from_delimited_files(bank_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
count,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000
mean,40.040212,257.335205,2.561730,962.174780,0.174780,0.076228,93.574243,-40.518680,3.615654,5166.859608
std,10.432313,257.331700,2.763646,187.646785,0.496503,1.572242,0.578636,4.623004,1.735748,72.208448
min,17.000000,0.000000,1.000000,0.000000,0.000000,-3.400000,92.201000,-50.800000,0.634000,4963.600000
25%,32.000000,102.000000,1.000000,999.000000,0.000000,-1.800000,93.075000,-42.700000,1.344000,5099.100000
50%,38.000000,179.000000,2.000000,999.000000,0.000000,1.100000,93.749000,-41.800000,4.857000,5191.000000
75%,47.000000,318.000000,3.000000,999.000000,0.000000,1.400000,93.994000,-36.400000,4.961000,5228.100000
max,98.000000,4918.000000,56.000000,999.000000,7.000000,1.400000,94.767000,-26.900000,5.045000,5228.100000


### Review the Dataset Result

You can peek the result of a TabularDataset at any range using `skip(i)` and `take(j).to_pandas_dataframe()`. Doing so evaluates only `j` records for all the steps in the TabularDataset, which makes it fast even against large datasets.

`TabularDataset` objects are composed of a list of transformation steps (optional).

In [7]:
dataset.take(10).to_pandas_dataframe()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,57,technician,married,high.school,no,no,yes,cellular,may,mon,...,1,999,1,failure,-1.8,92.893,-46.2,1.299,5099.1,no
1,55,unknown,married,unknown,unknown,yes,no,telephone,may,thu,...,2,999,0,nonexistent,1.1,93.994,-36.4,4.860,5191.0,no
2,33,blue-collar,married,basic.9y,no,no,no,cellular,may,fri,...,1,999,1,failure,-1.8,92.893,-46.2,1.313,5099.1,no
3,36,admin.,married,high.school,no,no,no,telephone,jun,fri,...,4,999,0,nonexistent,1.4,94.465,-41.8,4.967,5228.1,no
4,27,housemaid,married,high.school,no,yes,no,cellular,jul,fri,...,2,999,0,nonexistent,1.4,93.918,-42.7,4.963,5228.1,no
5,58,retired,married,professional.course,no,yes,yes,cellular,jul,fri,...,1,999,0,nonexistent,1.4,93.918,-42.7,4.962,5228.1,no
6,48,services,married,high.school,unknown,yes,no,telephone,may,wed,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.856,5191.0,no
7,51,admin.,divorced,university.degree,unknown,yes,no,cellular,aug,thu,...,7,999,0,nonexistent,1.4,93.444,-36.1,4.962,5228.1,no
8,24,entrepreneur,married,university.degree,no,yes,yes,telephone,jun,wed,...,4,999,0,nonexistent,1.4,94.465,-41.8,4.962,5228.1,no
9,36,technician,divorced,professional.course,no,yes,yes,cellular,jul,mon,...,4,999,0,nonexistent,1.4,93.918,-42.7,4.962,5228.1,no


## Train
This creates a general AutoML settings object.
**Udacity notes:** These inputs must match what was used when training in the portal. `time_column_name` has to be `cnt` for example.

In [8]:
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 4,
    "primary_metric" : 'accuracy',
    "n_cross_validations": 5
}
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="y",  
                             featurization= 'auto',
                             path = project_folder,
                             enable_early_stopping= True,
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

#### Create Pipeline and AutoMLStep

You can define outputs for the AutoMLStep using TrainingOutput.

In [9]:
from azureml.pipeline.core import PipelineData, TrainingOutput

ds = ws.get_default_datastore()
metrics_output_name = 'metrics_output'
best_model_output_name = 'best_model_output'

metrics_data = PipelineData(name='metrics_data',
                           datastore=ds,
                           pipeline_output_name=metrics_output_name,
                           training_output=TrainingOutput(type='Metrics'))
model_data = PipelineData(name='model_data',
                           datastore=ds,
                           pipeline_output_name=best_model_output_name,
                           training_output=TrainingOutput(type='Model'))

Create an AutoMLStep.

In [10]:
automl_step = AutoMLStep(
    name='automl_module',
    automl_config=automl_config,
    outputs=[metrics_data, model_data],
    allow_reuse=True)

In [11]:
from azureml.pipeline.core import Pipeline
pipeline = Pipeline(
    description="pipeline_with_automlstep",
    workspace=ws,    
    steps=[automl_step])

In [12]:
pipeline_run = experiment.submit(pipeline)

Created step automl_module [c32f8004][9687b66a-89c7-48c8-b6a8-a68403d40c7b], (This step will run and generate new outputs)
Submitted PipelineRun 48c18109-9502-4298-bd89-515c0c9f3cdf
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/sayed-bankmarketing-exp/runs/48c18109-9502-4298-bd89-515c0c9f3cdf?wsid=/subscriptions/3e42d11f-d64d-4173-af9b-12ecaa1030b3/resourcegroups/aml-quickstarts-135932/workspaces/quick-starts-ws-135932


In [13]:
from azureml.widgets import RunDetails
RunDetails(pipeline_run).show()

_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

In [14]:
pipeline_run.wait_for_completion()

PipelineRunId: 48c18109-9502-4298-bd89-515c0c9f3cdf
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/sayed-bankmarketing-exp/runs/48c18109-9502-4298-bd89-515c0c9f3cdf?wsid=/subscriptions/3e42d11f-d64d-4173-af9b-12ecaa1030b3/resourcegroups/aml-quickstarts-135932/workspaces/quick-starts-ws-135932
PipelineRun Status: Running


StepRunId: f12badf2-f95f-497d-8c0d-8808f9f30d2c
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/sayed-bankmarketing-exp/runs/f12badf2-f95f-497d-8c0d-8808f9f30d2c?wsid=/subscriptions/3e42d11f-d64d-4173-af9b-12ecaa1030b3/resourcegroups/aml-quickstarts-135932/workspaces/quick-starts-ws-135932
StepRun( automl_module ) Status: Running

StepRun(automl_module) Execution Summary
StepRun( automl_module ) Status: Finished
{'runId': 'f12badf2-f95f-497d-8c0d-8808f9f30d2c', 'target': 'sayed-cluster', 'status': 'Completed', 'startTimeUtc': '2021-01-24T21:15:57.44557Z', 'endTimeUtc': '2021-01-24T22:03:37.959808Z', 'properties': {'azur

'Finished'

## Examine Results

### Retrieve the metrics of all child runs
Outputs of above run can be used as inputs of other steps in pipeline. In this tutorial, we will examine the outputs by retrieve output data and running some tests.

In [15]:
metrics_output = pipeline_run.get_pipeline_output(metrics_output_name)
num_file_downloaded = metrics_output.download('.', show_progress=True)

Downloaded azureml/f12badf2-f95f-497d-8c0d-8808f9f30d2c/metrics_data, 1 files out of an estimated total of 1


In [16]:
import json
with open(metrics_output._path_on_datastore) as f:
    metrics_output_result = f.read()
    
deserialized_metrics_output = json.loads(metrics_output_result)
df = pd.DataFrame(deserialized_metrics_output)
df

,f12badf2-f95f-497d-8c0d-8808f9f30d2c_15,f12badf2-f95f-497d-8c0d-8808f9f30d2c_41,f12badf2-f95f-497d-8c0d-8808f9f30d2c_43,f12badf2-f95f-497d-8c0d-8808f9f30d2c_1,f12badf2-f95f-497d-8c0d-8808f9f30d2c_19,f12badf2-f95f-497d-8c0d-8808f9f30d2c_35,f12badf2-f95f-497d-8c0d-8808f9f30d2c_25,f12badf2-f95f-497d-8c0d-8808f9f30d2c_31,f12badf2-f95f-497d-8c0d-8808f9f30d2c_42,f12badf2-f95f-497d-8c0d-8808f9f30d2c_8,...,f12badf2-f95f-497d-8c0d-8808f9f30d2c_17,f12badf2-f95f-497d-8c0d-8808f9f30d2c_32,f12badf2-f95f-497d-8c0d-8808f9f30d2c_23,f12badf2-f95f-497d-8c0d-8808f9f30d2c_29,f12badf2-f95f-497d-8c0d-8808f9f30d2c_3,f12badf2-f95f-497d-8c0d-8808f9f30d2c_34,f12badf2-f95f-497d-8c0d-8808f9f30d2c_9,f12badf2-f95f-497d-8c0d-8808f9f30d2c_33,f12badf2-f95f-497d-8c0d-8808f9f30d2c_40,f12badf2-f95f-497d-8c0d-8808f9f30d2c_7
AUC_micro,[0.9737963622631429],[0.9801881385554514],[0.9802980259325184],[0.9806089467418561],[0.9644327060129271],[0.8879514415781488],[0.8879514415781488],[0.9721957142955828],[0.9641756926966641],[0.9714106488655961],...,[0.9587994593362362],[0.9806882548396085],[0.9799142260425853],[0.8879514415781488],[0.9657752929554826],[0.9804033379309708],[0.9750174932819995],[0.9756230182761852],[0.9762561129775422],[0.9800039628719655]
f1_score_macro,[0.5614778178574683],[0.7667005417976915],[0.7633204954496321],[0.7606149023199388],[0.4703248283762755],[0.4703248283762755],[0.4703248283762755],[0.69199277346886],[0.5396600295040513],[0.6368169457670512],...,[0.4703248283762755],[0.76531234625362],[0.7429776694256687],[0.4703248283762755],[0.4703248283762755],[0.754177917637638],[0.6704040231096142],[0.4703248283762755],[0.744808137758876],[0.7605794615939807]
matthews_correlation,[0.24558052522900678],[0.5370228063613797],[0.5310342240388785],[0.5283949538267647],[0.0],[0.0],[0.0],[0.42554467089064224],[0.229806927926384],[0.3573067137174021],...,[0.0],[0.5346541203230035],[0.49745127344480655],[0.0],[0.0],[0.5164039905978666],[0.39040297971515703],[0.0],[0.49685093695719],[0.524115222827228]
precision_score_macro,[0.8108632211599309],[0.792315281021075],[0.7920191520154423],[0.798786649757224],[0.4439757207890743],[0.4439757207890743],[0.4439757207890743],[0.8019971481318141],[0.8674593811299582],[0.8191044240017918],...,[0.4439757207890743],[0.7927246034651576],[0.7923217465595622],[0.4439757207890743],[0.4439757207890743],[0.7943413566727022],[0.791956144727526],[0.4439757207890743],[0.7816368152210142],[0.7830528711939717]
average_precision_score_weighted,[0.9445712950343926],[0.954244686927072],[0.9544340215442876],[0.9559275656602093],[0.9252626630956244],[0.8010198005438876],[0.8010198005438876],[0.9422230270541154],[0.9231906417956507],[0.937846147035667],...,[0.9169646613902243],[0.9551834358619609],[0.9539984290100796],[0.8010198005438876],[0.9253959958679028],[0.9547950294790933],[0.9449486450471511],[0.9522478063249695],[0.946409062420965],[0.9537778328527711]
average_precision_score_micro,[0.9748086708035315],[0.9809975534928818],[0.981132015417878],[0.9814408813866381],[0.9656546055144883],[0.844485621061018],[0.844485621061018],[0.9731662853903261],[0.9597763765847903],[0.9695781278652793],...,[0.9601433153310135],[0.9815287301407991],[0.9808006129474964],[0.844485621061018],[0.9668211415120063],[0.981109664315768],[0.9760083763364467],[0.9767372609924768],[0.9768937880760025],[0.9807704810500646]
recall_score_micro,[0.8950227617602428],[0.9142943854324734],[0.9138391502276176],[0.9149317147192717],[0.8879514415781486],[0.8879514415781486],[0.8879514415781486],[0.9071623672230652],[0.8946282245827011],[0.9027921092564493],...,[0.8879514415781486],[0.9142336874051592],[0.9115022761760244],[0.8879514415781486],[0.8879514415781486],[0.9132321699544764],[0.903641881638847],[0.8879514415781486],[0.9095295902883157],[0.9114415781487102]
AUC_weighted,[0.9303137106697402],[0.945801227702338],[0.9463878458020446],[0.9469943280414782],[0.8843710693277179],[0.5],[0.5],[0.9132004151094882],[0.8813005038125418],[0.9148716190374383],...,[0.856060

### Retrieve the Best Model

In [17]:
# Retrieve best model from Pipeline Run
best_model_output = pipeline_run.get_pipeline_output(best_model_output_name)
num_file_downloaded = best_model_output.download('.', show_progress=True)

Downloaded azureml/f12badf2-f95f-497d-8c0d-8808f9f30d2c/model_data, 1 files out of an estimated total of 1


In [18]:
import pickle

with open(best_model_output._path_on_datastore, "rb" ) as f:
    best_model = pickle.load(f)
best_model

PipelineWithYTransformations(Pipeline={'memory': None,
                                       'steps': [('datatransformer',
                                                  DataTransformer(enable_dnn=None,
                                                                  enable_feature_sweeping=None,
                                                                  feature_sweeping_config=None,
                                                                  feature_sweeping_timeout=None,
                                                                  featurization_config=None,
                                                                  force_text_dnn=None,
                                                                  is_cross_validation=None,
                                                                  is_onnx_compatible=None,
                                                                  logger=None,
                                                              

In [19]:
best_model.steps

[('datatransformer',
  DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                  feature_sweeping_config=None, feature_sweeping_timeout=None,
                  featurization_config=None, force_text_dnn=None,
                  is_cross_validation=None, is_onnx_compatible=None, logger=None,
                  observer=None, task=None, working_dir=None)),
 ('prefittedsoftvotingclassifier',
  PreFittedSoftVotingClassifier(classification_labels=None,
                                estimators=[('36',
                                             Pipeline(memory=None,
                                                      steps=[('sparsenormalizer',
                                                              <azureml.automl.runtime.shared.model_wrappers.SparseNormalizer object at 0x7efeed23fa58>),
                                                             ('xgboostclassifier',
                                                              XGBoostClassifier(base_score=0

## Publish and run from REST endpoint

Run the following code to publish the pipeline to your workspace. In your workspace in the portal, you can see metadata for the pipeline including run history and durations. You can also run the pipeline manually from the portal.

Additionally, publishing the pipeline enables a REST endpoint to rerun the pipeline from any HTTP library on any platform.


In [20]:
# Ensure that the workspace is once again available
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

quick-starts-ws-135932
aml-quickstarts-135932
southcentralus
3e42d11f-d64d-4173-af9b-12ecaa1030b3


In [21]:
# NOTE: update these to match your existing experiment name and a previous experiment


experiment = Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
sayed-bankmarketing-exp,quick-starts-ws-135932,Link to Azure Machine Learning studio,Link to Documentation


In [22]:
published_pipeline = pipeline_run.publish_pipeline(
    name="Bankmarketing Train by sayed", description="Training bankmarketing pipeline by sayed ahmed", version="1.0")

published_pipeline


Name,Id,Status,Endpoint
Bankmarketing Train by sayed,77e2ec70-7d88-4926-91dd-d2737b9dcfa5,Active,REST Endpoint


Authenticate once again, to retrieve the `auth_header` so that the endpoint can be used

In [23]:
from azureml.core.authentication import InteractiveLoginAuthentication

interactive_auth = InteractiveLoginAuthentication()
auth_header = interactive_auth.get_authentication_header()



Get the REST url from the endpoint property of the published pipeline object. You can also find the REST url in your workspace in the portal. Build an HTTP POST request to the endpoint, specifying your authentication header. Additionally, add a JSON payload object with the experiment name and the batch size parameter. As a reminder, the process_count_per_node is passed through to ParallelRunStep because you defined it is defined as a PipelineParameter object in the step configuration.

Make the request to trigger the run. Access the Id key from the response dict to get the value of the run id.


In [24]:
import requests

rest_endpoint = published_pipeline.endpoint
response = requests.post(rest_endpoint, 
                         headers=auth_header, 
                         json={"ExperimentName": "pipeline-bankmarketing-rest-endpoint"}
                        )

In [25]:
try:
    response.raise_for_status()
except Exception:    
    raise Exception("Received bad response from the endpoint: {}\n"
                    "Response Code: {}\n"
                    "Headers: {}\n"
                    "Content: {}".format(rest_endpoint, response.status_code, response.headers, response.content))

run_id = response.json().get('Id')
print('Submitted pipeline run: ', run_id)

Submitted pipeline run:  07204da6-8b94-4680-9388-dceb5c307053


Use the run id to monitor the status of the new run. This will take another 10-15 min to run and will look similar to the previous pipeline run, so if you don't need to see another pipeline run, you can skip watching the full output.

In [26]:
from azureml.pipeline.core.run import PipelineRun
from azureml.widgets import RunDetails

published_pipeline_run = PipelineRun(ws.experiments["pipeline-bankmarketing-rest-endpoint"], run_id)
RunDetails(published_pipeline_run).show()

_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

In [27]:
published_pipeline_run

Experiment,Id,Type,Status,Details Page,Docs Page
pipeline-bankmarketing-rest-endpoint,07204da6-8b94-4680-9388-dceb5c307053,azureml.PipelineRun,Completed,Link to Azure Machine Learning studio,Link to Documentation
